In [32]:
import os
import re
from collections import defaultdict

In [33]:
# Preprocessing function
def preprocess(text):
    return re.findall(r'\b\w+\b', text.lower())

In [34]:
# Load documents
def load_documents(folder_path):
    docs = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            with open(os.path.join(folder_path, filename), 'r') as file:
                docs[filename] = preprocess(file.read())
    return docs

In [35]:
# Load queries
def load_queries(query_file_path):
    with open(query_file_path, 'r') as file:
        return [line.strip() for line in file.readlines()]

In [36]:
# Compute term frequencies and document frequencies
def compute_statistics(docs):
    doc_count = len(docs)
    term_doc_freq = defaultdict(int)
    term_freq = defaultdict(lambda: defaultdict(int))

    for doc_id, words in docs.items():
        word_set = set(words)
        for word in words:
            term_freq[doc_id][word] += 1
        for word in word_set:
            term_doc_freq[word] += 1

    return term_freq, term_doc_freq, doc_count

In [37]:
# Compute relevance probabilities using BIM
def compute_relevance_prob(query, term_freq, term_doc_freq, doc_count):
    scores = {}
    for doc_id in term_freq:
        score = 1.0
        for term in query:
            tf = term_freq[doc_id].get(term, 0)
            df = term_doc_freq.get(term, 0)
            p_term_given_relevant = (tf + 1) / (sum(term_freq[doc_id].values()) + len(term_doc_freq))
            p_term_given_not_relevant = (df + 1) / (doc_count - df + len(term_doc_freq))
            score *= (p_term_given_relevant / p_term_given_not_relevant)
        scores[doc_id] = score
    return scores

In [38]:
# Main retrieval function
def retrieve_documents_and_result(trump_speeches_path, query_file_path, output_file_name):
    docs = load_documents(trump_speeches_path)
    queries = load_queries(query_file_path)

    term_freq, term_doc_freq, doc_count = compute_statistics(docs)

    with open(output_file_name, 'w') as results_file:
        for query in queries:
            query_terms = preprocess(query)
            scores = compute_relevance_prob(query_terms, term_freq, term_doc_freq, doc_count)
            ranked_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)
            results_file.write(f"Query: {query}\n")
            for doc_id, score in ranked_docs:
                results_file.write(f"Document: {doc_id}, Score: {score:.4f}\n")
            results_file.write("\n")
    print(f"Results written to {output_file_name}")

In [39]:
# Main function
def main():
    trump_speeches_path = 'Speechs'
    query_file_path =  './query/query1.txt'
    output_file_name = 'results.txt'
    retrieve_documents_and_result(trump_speeches_path, query_file_path, output_file_name)

if __name__ == "__main__":
    main()

Results written to results.txt
